In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card/application_data.csv
/kaggle/input/credit-card/previous_application.csv
/kaggle/input/credit-card/columns_description.csv


In [2]:
!pip install imbalanced-learn

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [6]:
df=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [7]:
df.shape

(284807, 31)

In [17]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

In [20]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [21]:
logclass=LogisticRegression()
grid={"C":(10.0)**np.arange(-2,3,10),'penalty':['l1','l2']}
cv=KFold(n_splits=5,shuffle=False)

In [23]:
clg=GridSearchCV(logclass,param_grid=grid,cv=cv,n_jobs=-1,scoring='f1_macro')

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
y=df['Class']

In [26]:
cols=list(df.columns)

In [29]:
cols.remove("Class")
cols

['Time',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount']

In [30]:
X=df[cols]

In [32]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,stratify=y)

In [34]:
from sklearn.metrics import classification_report

In [33]:
clg.fit(xtrain,ytrain)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([0.01]), 'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [35]:
ypred=clg.predict(xtest)
print(confusion_matrix(ytest,ypred))
print(classification_report(ytest,ypred))

[[56834    30]
 [   31    67]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.69      0.68      0.69        98

    accuracy                           1.00     56962
   macro avg       0.85      0.84      0.84     56962
weighted avg       1.00      1.00      1.00     56962



In [36]:
from sklearn.ensemble import RandomForestClassifier
rfcls=RandomForestClassifier()
rfcls.fit(xtrain,ytrain)

RandomForestClassifier()

In [37]:
ypred=rfcls.predict(xtest)
print(confusion_matrix(ytest,ypred))
print(classification_report(ytest,ypred))

[[56860     4]
 [   21    77]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.95      0.79      0.86        98

    accuracy                           1.00     56962
   macro avg       0.98      0.89      0.93     56962
weighted avg       1.00      1.00      1.00     56962



In [38]:
#random forst using class weight
classweight={0:1,1:100}
from sklearn.ensemble import RandomForestClassifier
rfcls=RandomForestClassifier(class_weight=classweight)
rfcls.fit(xtrain,ytrain)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [39]:
ypred=rfcls.predict(xtest)
print(confusion_matrix(ytest,ypred))
print(classification_report(ytest,ypred))

[[56861     3]
 [   21    77]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.96      0.79      0.87        98

    accuracy                           1.00     56962
   macro avg       0.98      0.89      0.93     56962
weighted avg       1.00      1.00      1.00     56962



In [46]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek

In [41]:
from collections import Counter

In [43]:
rosampler=RandomOverSampler(0.75)
xtrainos,ytrainos=rosampler.fit_sample(xtrain,ytrain)
print("the terms ",Counter(ytrain))
print("the terms ",Counter(ytrainos))

/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:638: FutureWarning: Pass sampling_strategy=0.75 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


the terms  Counter({0: 227451, 1: 394})
the terms  Counter({0: 227451, 1: 170588})


In [44]:
from sklearn.ensemble import RandomForestClassifier
rfcls=RandomForestClassifier()
rfcls.fit(xtrainos,ytrainos)

RandomForestClassifier()

In [45]:
ypred=rfcls.predict(xtest)
print(confusion_matrix(ytest,ypred))
print(classification_report(ytest,ypred))

[[56858     6]
 [   19    79]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.93      0.81      0.86        98

    accuracy                           1.00     56962
   macro avg       0.96      0.90      0.93     56962
weighted avg       1.00      1.00      1.00     56962



In [ ]:
rosampler=SMOTETomek(0.75)
xtrainsm,ytrainsm=rosampler.fit_sample(xtrain,ytrain)
print("the terms ",Counter(ytrain))
print("the terms ",Counter(ytrainsm))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfcls=RandomForestClassifier()
rfcls.fit(xtrainsm,ytrainsm)

In [ ]:
ypred=rfcls.predict(xtest)
print(confusion_matrix(ytest,ypred))
print(classification_report(ytest,ypred))